# Tracking Numerics in Jax / Flax
Simple MNIST MLP example.

In [1]:
# Install dependencies if needed
!pip install jax flax optax


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Copyright 2018 The JAX Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Modified by Graphcore Ltd 2024.


"""Datasets used in examples."""


import array
import gzip
import os
import pickle
import struct
import tarfile
import urllib.request
from os import path

import numpy as np

_DATA = "/tmp/jax_example_data/"


def _download(url, filename):
    """Download a url to a file in the JAX data temp directory."""
    if not path.exists(_DATA):
        os.makedirs(_DATA)
    out_file = path.join(_DATA, filename)
    if not path.isfile(out_file):
        urllib.request.urlretrieve(url, out_file)
        print(f"downloaded {url} to {_DATA}")


def _partial_flatten(x):
    """Flatten all but the first dimension of an ndarray."""
    return np.reshape(x, (x.shape[0], -1))


def _one_hot(x, k, dtype=np.float32):
    """Create a one-hot encoding of x of size k."""
    return np.array(x[:, None] == np.arange(k), dtype)


def _unzip(file):
    file = tarfile.open(file)
    file.extractall(_DATA)
    file.close()
    return


def _unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


def mnist_raw():
    """Download and parse the raw MNIST dataset."""
    # CVDF mirror of http://yann.lecun.com/exdb/mnist/
    base_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"

    def parse_labels(filename):
        with gzip.open(filename, "rb") as fh:
            _ = struct.unpack(">II", fh.read(8))
            return np.array(array.array("B", fh.read()), dtype=np.uint8)

    def parse_images(filename):
        with gzip.open(filename, "rb") as fh:
            _, num_data, rows, cols = struct.unpack(">IIII", fh.read(16))
            return np.array(array.array("B", fh.read()), dtype=np.uint8).reshape(num_data, rows, cols)

    for filename in [
        "train-images-idx3-ubyte.gz",
        "train-labels-idx1-ubyte.gz",
        "t10k-images-idx3-ubyte.gz",
        "t10k-labels-idx1-ubyte.gz",
    ]:
        _download(base_url + filename, filename)

    train_images = parse_images(path.join(_DATA, "train-images-idx3-ubyte.gz"))
    train_labels = parse_labels(path.join(_DATA, "train-labels-idx1-ubyte.gz"))
    test_images = parse_images(path.join(_DATA, "t10k-images-idx3-ubyte.gz"))
    test_labels = parse_labels(path.join(_DATA, "t10k-labels-idx1-ubyte.gz"))

    return train_images, train_labels, test_images, test_labels


def mnist(permute_train=False):
    """Download, parse and process MNIST data to unit scale and one-hot labels."""
    train_images, train_labels, test_images, test_labels = mnist_raw()

    train_images = _partial_flatten(train_images) / np.float32(255.0)
    test_images = _partial_flatten(test_images) / np.float32(255.0)
    train_labels = _one_hot(train_labels, 10)
    test_labels = _one_hot(test_labels, 10)

    if permute_train:
        perm = np.random.RandomState(0).permutation(train_images.shape[0])
        train_images = train_images[perm]
        train_labels = train_labels[perm]

    return train_images, train_labels, test_images, test_labels

In [3]:
import time
from functools import partial
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax import linen as nn  # type:ignore
from tandv.track.jax import track

In [4]:



# from jax.scipy.special import logsumexp


def logsumexp(a, axis=None, keepdims=False):
    from jax import lax

    dims = (axis,)
    amax = jnp.max(a, axis=dims, keepdims=keepdims)
    # FIXME: not proper scale propagation, introducing NaNs
    # amax = lax.stop_gradient(lax.select(jnp.isfinite(amax), amax, lax.full_like(amax, 0)))
    amax = lax.stop_gradient(amax)
    out = lax.sub(a, amax)
    out = lax.exp(out)
    out = lax.add(lax.log(jnp.sum(out, axis=dims, keepdims=keepdims)), amax)
    return out


class MLP(nn.Module):
    """A simple 3 layers MLP model."""

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=512, use_bias=True)(x)
        x = nn.relu(x)
        x = nn.Dense(features=512, use_bias=True)(x)
        x = nn.relu(x)
        x = nn.Dense(features=10, use_bias=True)(x)
        logprobs = x - logsumexp(x, axis=1, keepdims=True)
        return logprobs


def loss(model, params, batch):
    inputs, targets = batch
    preds = model.apply(params, inputs)
    # targets = jsa.lax.rebalance(targets, np.float32(1 / 8))
    return -jnp.mean(jnp.sum(preds * targets, axis=1))


def accuracy(model, params, batch):
    inputs, targets = batch
    target_class = jnp.argmax(targets, axis=1)
    preds = model.apply(params, inputs)
    predicted_class = jnp.argmax(preds, axis=1)
    return jnp.mean(predicted_class == target_class)


def update(model, optimizer, model_state, opt_state, batch):
    grads = jax.grad(partial(loss, model))(model_state, batch)
    # Optimizer update (state & gradients).
    updates, opt_state = optimizer.update(grads, opt_state, model_state)
    model_state = optax.apply_updates(model_state, updates)
    return model_state, opt_state, grads

In [6]:
step_size = 0.001
num_epochs = 3
batch_size = 128
key = jax.random.PRNGKey(42)
use_scalify: bool = True

training_dtype = np.dtype(np.float16)
optimizer_dtype = np.dtype(np.float16)
scale_dtype = np.float32

train_images, train_labels, test_images, test_labels = mnist()
num_train = train_images.shape[0]
num_complete_batches, leftover = divmod(num_train, batch_size)
num_batches = num_complete_batches + bool(leftover)
mnist_img_size = train_images.shape[-1]

def data_stream():
    rng = np.random.RandomState(0)
    while True:
        perm = rng.permutation(num_train)
        for i in range(num_batches):
            batch_idx = perm[i * batch_size : (i + 1) * batch_size]
            yield train_images[batch_idx], train_labels[batch_idx]

# Build model & initialize model parameters.
model = MLP()
model_state = model.init(key, np.zeros((batch_size, mnist_img_size), dtype=training_dtype))
# Optimizer & optimizer state.
opt = optax.adam(learning_rate=step_size, eps=2**-16)
opt_state = opt.init(model_state)


batches = data_stream()

with track(model_state=model_state,optimizer_state=opt_state,track_gradients=True) as tracker: # wrap training loop in TorchTracker context manager

    for epoch in range(num_epochs):
        start_time = time.time()

        for _ in range(num_batches):
            batch = next(batches)

            model_state, opt_state, grads = tracker.intercept(update,model,opt,model_state, opt_state, batch) # pass update fn & args into intercept method (capture activations & grads)
            tracker.step(model_state, opt_state, grads) # call step and pass pytrees for whatever you're tracking as args



The output LogFrame is available at: /Users/colmb/numerics-vis/example-notebooks/tandv/damp-manuscript/final_logframe.pkl


In [13]:
from tandv.track.common import read_pickle
read_pickle('/Users/colmb/numerics-vis/example-notebooks/tandv/damp-manuscript/final_logframe.pkl')

metadata                                              \
          step     name                               type   
0            0  Dense_0                               None   
1            0  Dense_1                               None   
2            0  Dense_2                               None   
3            0  Dense_0                               None   
4            0  Dense_1                               None   
...        ...      ...                                ...   
28135     1406  Dense_1  <class 'flax.linen.linear.Dense'>   
28136     1406  Dense_0  <class 'flax.linen.linear.Dense'>   
28137     1406  Dense_0                               None   
28138     1406  Dense_1                               None   
28139     1406  Dense_2                               None   

                                                                   \
                                             tensor_type    dtype   
0                                                Weights  float32   
1                                                Weights  float32   
2                                                Weights  float32   
3      Optimiser_State                               ...  float32   
4      Optimiser_State                               ...  float32   
...                                                  ...      ...   
28135                                           Gradient  float32   
28136                                           Gradient  float32   
28137                                   Weight_Gradients  float32   
28138                                   Weight_Gradients  float32   
28139                                   Weight_Gradients  float32   

      scalar_stats                                                  ...  \
           max_abs          mean  mean_abs       min_abs       rm2  ...   
0         0.081203  1.872651e-05  0.029328  4.747923e-07  0.035703  ...   
1         0.100479 -6.272813e-05  0.036273  2.722679e-07  0.044156  ...   
2         0.100483  5.343384e-04  0.036234  9.199790e-06  0.044167  ...   
3         0.000000  0.000000e+00  0.000000  0.000000e+00  0.000000  ...   
4         0.000000  0.000000e+00  0.000000  0.000000e+00  0.000000  ...   
...            ...           ...       ...           ...       ...  ...   
28135     0.002142  3.508286e-08  0.000014  0.000000e+00  0.000096  ...   
28136     0.014246  2.012506e-06  0.000022  0.000000e+00  0.000248  ...   
28137     0.015556  1.756449e-05  0.000177  0.000000e+00  0.000666  ...   
28138     0.003809 -8.185178e-08  0.000137  0.000000e+00  0.000288  ...   
28139     0.018990 -1.377771e-10  0.001873  0.000000e+00  0.003539  ...   

      exponent_counts                              
                    8  9 10 11 12 13 14 15 16 inf  
0                   0  0  0  0  0  0  0  0  0   0  
1                   0  0  0  0  0  0  0  0  0   0  
2                   0  0  0  0  0  0  0  0  0   0  
3                   0  0  0  0  0  0  0  0  0   0  
4                   0  0  0  0  0  0  0  0  0   0  
...               ... .. .. .. .. .. .. .. ..  ..  
28135               0  0  0  0  0  0  0  0  0   0  
28136               0  0  0  0  0  0  0  0  0   0  
28137               0  0  0  0  0  0  0  0  0   0  
28138               0  0  0  0  0  0  0  0  0   0  
28139               0  0  0  0  0  0  0  0  0   0  

[28140 rows x 48 columns]